# * VINSIGHT : SubBase, 60DPD, Reported SubBase
    Reported SubBase Pre & Post มาจากพี่เอ้รัน Script พี่หนุ่ย (ไม่รวม FTTx)
    นอกนั้นมาจากพี่เอก รวมถึง SubBase, 60DPD

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Import Transaction

In [2]:
''' Execute transaction '''


# Input parameter
v_start_date = 20240101
print(f'\nParameter input...')
print(f'   -> v_start_date: {v_start_date}')

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B1S000702' --Prepaid SubBase
        , 'TB1S000702' --Prepaid SubBase : TMH
        , 'DB1S000702' --Prepaid SubBase : DTAC
        , 'B1S000700' --Prepaid Reported SubBase
        , 'TB1S000700' --Prepaid Reported SubBase : TMH
        , 'DB1S000700' --Prepaid Reported SubBase : DTAC
        
        , 'B2S010602' --Postpaid SubBase B2C
        , 'DB2S010602' --Postpaid SubBase B2C : DTAC
        , 'TB2S010602' --Postpaid SubBase B2C : TMH
        , 'B2S010601' --Postpaid 60DPD B2C
        , 'TB2S010601' --Postpaid 60DPD B2C : TMH
        , 'DB2S010601' --Postpaid 60DPD B2C : DTAC
        , 'B2S010603' --Postpaid Gain/Loss 60DPD B2C
        , 'TB2S010603' --Postpaid Gain/Loss 60DPD B2C : TMH
        , 'DB2S010603' --Postpaid Gain/Loss 60DPD B2C : DTAC
        , 'B2S010600' --Postpaid Reported SubBase B2C
        , 'DB2S010600' --Postpaid Reported SubBase B2C : DTAC
        , 'TB2S010600' --Postpaid Reported SubBase B2C : TMH
        
        , 'TB3S020603' --FTTx 60DPD
        , 'TB3S020606' --FTTx Gain/Loss 60DPD
        , 'TB3S000600' --FTTx Reported SubBase
        , 'TB3S020604' --FTTx SubBase
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    --AND TM_KEY_DAY IN (20240131, 20240229, 20240331, 20240430, 20240531, 20240630, 20240731, 20240831, 20240930, 20241031, 20241130, 20241231, 20250131, 20250228, 20250331) -->> Year 2024
    --AND TM_KEY_DAY IN (20250131, 20250228, 20250331, 20250430, 20250531) -->> Year 2025
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
    --ORDER BY TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD
""")


try:
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    print(f'\nDataFrame: {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')
    
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...
   -> v_start_date: 20240101

Data as of 2025-06-11, 10:27:19

DataFrame: 8959 rows, 16 columns


In [3]:
''' Automate Currently Period '''

curr_yr = chk_src_df['TM_KEY_YR'].max()
prev_yr = chk_src_df['TM_KEY_YR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_qtr = chk_src_df['TM_KEY_QTR'].max()
prev_qtr = chk_src_df['TM_KEY_QTR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = chk_src_df['TM_KEY_MTH'].drop_duplicates().sort_values().shift().max().astype(int)

curr_wk = chk_src_df['TM_KEY_WK'].max()
prev_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift().max().astype(int)
last_3_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift(3).max().astype(int)

# curr_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].max().astype(int)
# prev_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].drop_duplicates().shift().max().astype(int)

In [4]:
# v_metric_list = [
#     'B1S000702' #Prepaid SubBase
#     , 'TB1S000702' #Prepaid SubBase : TMH
#     , 'DB1S000702' #Prepaid SubBase : DTAC
# 	, 'B1S000700' #Prepaid Reported SubBase
# 	, 'TB1S000700' #Prepaid Reported SubBase : TMH
# 	, 'DB1S000700' #Prepaid Reported SubBase : DTAC

# 	, 'B2S010602' #Postpaid SubBase B2C
#     , 'TB2S010602' #Postpaid SubBase B2C : TMH
# 	, 'DB2S010602' #Postpaid SubBase B2C : DTAC
# 	, 'B2S010601' #Postpaid 60DPD B2C
#     , 'TB2S010601' #Postpaid 60DPD B2C : TMH
# 	, 'DB2S010601' #Postpaid 60DPD B2C : DTAC
# 	, 'B2S010603' #Postpaid Gain/Loss 60DPD B2C
#     , 'TB2S010603' #Postpaid Gain/Loss 60DPD B2C : TMH
# 	, 'DB2S010603' #Postpaid Gain/Loss 60DPD B2C : DTAC
# 	, 'B2S010600' #Postpaid Reported SubBase B2C
#     , 'TB2S010600' #Postpaid Reported SubBase B2C : TMH
# 	, 'DB2S010600' #Postpaid Reported SubBase B2C : DTAC
	
#     , 'TB3S020604' #FTTx SubBase
# 	, 'TB3S020603' #FTTx 60DPD
# 	, 'TB3S020606' #FTTx Gain/Loss 60DPD
# 	, 'TB3S000600' #FTTx Reported SubBase
#     ]

## Prepaid by Period

In [5]:
''' Prepaid : SubBase Yearly '''

v_metric_list = [
    'B1S000702' #Prepaid SubBase
    , 'TB1S000702' #Prepaid SubBase : TMH
    , 'DB1S000702' #Prepaid SubBase : DTAC
	, 'B1S000700' #Prepaid Reported SubBase
	, 'TB1S000700' #Prepaid Reported SubBase : TMH
	, 'DB1S000700' #Prepaid Reported SubBase : DTAC
    ]

pre_sub_yearly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_YR', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_YR
pre_sub_yearly_df = pre_sub_yearly_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']]
pre_sub_yearly_df = pre_sub_yearly_df.loc[pre_sub_yearly_df['METRIC_CD'].isin(v_metric_list)]

pre_sub_yearly_df['PRE_SB'] = np.where(pre_sub_yearly_df['METRIC_CD']=='B1S000702', pre_sub_yearly_df['P'], 0)
pre_sub_yearly_df['PRE_SB_T'] = np.where(pre_sub_yearly_df['METRIC_CD']=='TB1S000702', pre_sub_yearly_df['P'], 0)
pre_sub_yearly_df['PRE_SB_D'] = np.where(pre_sub_yearly_df['METRIC_CD']=='DB1S000702', pre_sub_yearly_df['P'], 0)

pre_sub_yearly_df['PRE_RPT_SB'] = np.where(pre_sub_yearly_df['METRIC_CD']=='B1S000700', pre_sub_yearly_df['P'], 0)
pre_sub_yearly_df['PRE_RPT_SB_T'] = np.where(pre_sub_yearly_df['METRIC_CD']=='TB1S000700', pre_sub_yearly_df['P'], 0)
pre_sub_yearly_df['PRE_RPT_SB_D'] = np.where(pre_sub_yearly_df['METRIC_CD']=='DB1S000700', pre_sub_yearly_df['P'], 0)

pre_sub_yearly_df = pre_sub_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max'
                                                        , 'PRE_SB':'sum', 'PRE_SB_T':'sum', 'PRE_SB_D':'sum'
                                                        , 'PRE_RPT_SB':'sum', 'PRE_RPT_SB_T':'sum', 'PRE_RPT_SB_D':'sum'})
pre_sub_yearly_df = pre_sub_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
pre_sub_yearly_df = pre_sub_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF'
                               , 'PRE_SB', 'PRE_SB_T', 'PRE_SB_D'
                               , 'PRE_RPT_SB', 'PRE_RPT_SB_T', 'PRE_RPT_SB_D']]

pre_sub_yearly_df_display = pre_sub_yearly_df.copy()
pre_sub_yearly_df_display['ACTUAL_AS_OF'] = pre_sub_yearly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = pre_sub_yearly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    pre_sub_yearly_df_display[col] = pre_sub_yearly_df_display[col].apply(lambda x: format(x, ',.0f'))
pre_sub_yearly_df_display

,TM_KEY_YR,PPN_TM,ACTUAL_AS_OF,PRE_SB,PRE_SB_T,PRE_SB_D,PRE_RPT_SB,PRE_RPT_SB_T,PRE_RPT_SB_D
0,2024,2025-06-11 01:34:02,20241231,0,"24,782,672",0,0,"21,220,694",0
1,2025,2025-06-11 01:34:02,20250609,0,"27,711,932",0,0,"23,803,763",0


In [6]:
''' Prepaid : SubBase Quarterly '''

v_metric_list = [
    'B1S000702' #Prepaid SubBase
    , 'TB1S000702' #Prepaid SubBase : TMH
    , 'DB1S000702' #Prepaid SubBase : DTAC
	, 'B1S000700' #Prepaid Reported SubBase
	, 'TB1S000700' #Prepaid Reported SubBase : TMH
	, 'DB1S000700' #Prepaid Reported SubBase : DTAC
    ]

pre_sub_quarterly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_QTR', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_QTR
pre_sub_quarterly_df = pre_sub_quarterly_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']]
pre_sub_quarterly_df = pre_sub_quarterly_df.loc[pre_sub_quarterly_df['METRIC_CD'].isin(v_metric_list)]

pre_sub_quarterly_df['PRE_SB'] = np.where(pre_sub_quarterly_df['METRIC_CD']=='B1S000702', pre_sub_quarterly_df['P'], 0)
pre_sub_quarterly_df['PRE_SB_T'] = np.where(pre_sub_quarterly_df['METRIC_CD']=='TB1S000702', pre_sub_quarterly_df['P'], 0)
pre_sub_quarterly_df['PRE_SB_D'] = np.where(pre_sub_quarterly_df['METRIC_CD']=='DB1S000702', pre_sub_quarterly_df['P'], 0)

pre_sub_quarterly_df['PRE_RPT_SB'] = np.where(pre_sub_quarterly_df['METRIC_CD']=='B1S000700', pre_sub_quarterly_df['P'], 0)
pre_sub_quarterly_df['PRE_RPT_SB_T'] = np.where(pre_sub_quarterly_df['METRIC_CD']=='TB1S000700', pre_sub_quarterly_df['P'], 0)
pre_sub_quarterly_df['PRE_RPT_SB_D'] = np.where(pre_sub_quarterly_df['METRIC_CD']=='DB1S000700', pre_sub_quarterly_df['P'], 0)

pre_sub_quarterly_df = pre_sub_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max'
                                                        , 'PRE_SB':'sum', 'PRE_SB_T':'sum', 'PRE_SB_D':'sum'
                                                        , 'PRE_RPT_SB':'sum', 'PRE_RPT_SB_T':'sum', 'PRE_RPT_SB_D':'sum'})
pre_sub_quarterly_df = pre_sub_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
pre_sub_quarterly_df = pre_sub_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF'
                               , 'PRE_SB', 'PRE_SB_T', 'PRE_SB_D'
                               , 'PRE_RPT_SB', 'PRE_RPT_SB_T', 'PRE_RPT_SB_D']]

pre_sub_quarterly_df_display = pre_sub_quarterly_df.copy()
pre_sub_quarterly_df_display['ACTUAL_AS_OF'] = pre_sub_quarterly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = pre_sub_quarterly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    pre_sub_quarterly_df_display[col] = pre_sub_quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))
pre_sub_quarterly_df_display

,TM_KEY_QTR,PPN_TM,ACTUAL_AS_OF,PRE_SB,PRE_SB_T,PRE_SB_D,PRE_RPT_SB,PRE_RPT_SB_T,PRE_RPT_SB_D
0,20241,2025-06-11 01:34:02,20240331,0,"25,748,719",0,0,"21,409,544",0
1,20242,2025-06-11 01:34:02,20240630,0,"25,384,868",0,0,"20,914,621",0
2,20243,2025-06-11 01:34:02,20240930,0,"24,977,745",0,0,"20,682,878",0
3,20244,2025-06-11 01:34:02,20241231,0,"24,782,672",0,0,"21,220,694",0
4,20251,2025-06-11 01:34:02,20250331,0,"24,578,672",0,0,"20,647,169",0
5,20252,2025-06-11 01:34:02,20250609,0,"27,711,932",0,0,"23,803,763",0


In [7]:
''' Prepaid : SubBase Monthly '''

# v_tm_key_yr = curr_yr
v_tm_key_yr = prev_yr

v_metric_list = [
    'B1S000702' #Prepaid SubBase
    , 'TB1S000702' #Prepaid SubBase : TMH
    , 'DB1S000702' #Prepaid SubBase : DTAC
	, 'B1S000700' #Prepaid Reported SubBase
	, 'TB1S000700' #Prepaid Reported SubBase : TMH
	, 'DB1S000700' #Prepaid Reported SubBase : DTAC
    ]

pre_sub_monthly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_MTH', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_MTH
pre_sub_monthly_df = pre_sub_monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_YR']>=v_tm_key_yr]
pre_sub_monthly_df = pre_sub_monthly_df.loc[pre_sub_monthly_df['METRIC_CD'].isin(v_metric_list)]

pre_sub_monthly_df['PRE_SB'] = np.where(pre_sub_monthly_df['METRIC_CD']=='B1S000702', pre_sub_monthly_df['P'], 0)
pre_sub_monthly_df['PRE_SB_T'] = np.where(pre_sub_monthly_df['METRIC_CD']=='TB1S000702', pre_sub_monthly_df['P'], 0)
pre_sub_monthly_df['PRE_SB_D'] = np.where(pre_sub_monthly_df['METRIC_CD']=='DB1S000702', pre_sub_monthly_df['P'], 0)

pre_sub_monthly_df['PRE_RPT_SB'] = np.where(pre_sub_monthly_df['METRIC_CD']=='B1S000700', pre_sub_monthly_df['P'], 0)
pre_sub_monthly_df['PRE_RPT_SB_T'] = np.where(pre_sub_monthly_df['METRIC_CD']=='TB1S000700', pre_sub_monthly_df['P'], 0)
pre_sub_monthly_df['PRE_RPT_SB_D'] = np.where(pre_sub_monthly_df['METRIC_CD']=='DB1S000700', pre_sub_monthly_df['P'], 0)

pre_sub_monthly_df = pre_sub_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max'
                                                        , 'PRE_SB':'sum', 'PRE_SB_T':'sum', 'PRE_SB_D':'sum'
                                                        , 'PRE_RPT_SB':'sum', 'PRE_RPT_SB_T':'sum', 'PRE_RPT_SB_D':'sum'})
pre_sub_monthly_df = pre_sub_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
pre_sub_monthly_df = pre_sub_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF'
                               , 'PRE_SB', 'PRE_SB_T', 'PRE_SB_D'
                               , 'PRE_RPT_SB', 'PRE_RPT_SB_T', 'PRE_RPT_SB_D']]

pre_sub_monthly_df_display = pre_sub_monthly_df.copy()
pre_sub_monthly_df_display['ACTUAL_AS_OF'] = pre_sub_monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = pre_sub_monthly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    pre_sub_monthly_df_display[col] = pre_sub_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
pre_sub_monthly_df_display

,TM_KEY_MTH,PPN_TM,ACTUAL_AS_OF,PRE_SB,PRE_SB_T,PRE_SB_D,PRE_RPT_SB,PRE_RPT_SB_T,PRE_RPT_SB_D
0,202402,2025-06-11 01:34:02,20240229,0,"25,840,436",0,0,"21,794,228",0
1,202403,2025-06-11 01:34:02,20240331,0,"25,748,719",0,0,"21,409,544",0
2,202404,2025-06-11 01:34:02,20240430,0,"25,446,381",0,0,"21,026,743",0
3,202405,2025-06-11 01:34:02,20240531,0,"25,329,884",0,0,"20,992,491",0
4,202406,2025-06-11 01:34:02,20240630,0,"25,384,868",0,0,"20,914,621",0
5,202407,2025-06-11 01:34:02,20240731,0,"25,008,838",0,0,"20,935,553",0
6,202408,2025-06-11 01:34:02,20240831,0,"25,090,909",0,0,"20,868,927",0
7,202409,2025-06-11 01:34:02,20240930,0,"24,977,745",0,0,"20,682,878",0
8,202410,2025-06-11 01:34:02,20241031,0,"24,836,634",0,0,"20,579,008",0
9,202411,2025-06-11 01:34:02,20241130,0,"24,637,406",0,0,"20,653,861",0


In [8]:
''' Prepaid : SubBase Weekly '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
    'B1S000702' #Prepaid SubBase
    , 'TB1S000702' #Prepaid SubBase : TMH
    , 'DB1S000702' #Prepaid SubBase : DTAC
	, 'B1S000700' #Prepaid Reported SubBase
	, 'TB1S000700' #Prepaid Reported SubBase : TMH
	, 'DB1S000700' #Prepaid Reported SubBase : DTAC
    ]

pre_sub_weekly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_WK', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_WK
pre_sub_weekly_df = pre_sub_weekly_df[['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth]
pre_sub_weekly_df = pre_sub_weekly_df.loc[pre_sub_weekly_df['METRIC_CD'].isin(v_metric_list)]

pre_sub_weekly_df['PRE_SB'] = np.where(pre_sub_weekly_df['METRIC_CD']=='B1S000702', pre_sub_weekly_df['P'], 0)
pre_sub_weekly_df['PRE_SB_T'] = np.where(pre_sub_weekly_df['METRIC_CD']=='TB1S000702', pre_sub_weekly_df['P'], 0)
pre_sub_weekly_df['PRE_SB_D'] = np.where(pre_sub_weekly_df['METRIC_CD']=='DB1S000702', pre_sub_weekly_df['P'], 0)

pre_sub_weekly_df['PRE_RPT_SB'] = np.where(pre_sub_weekly_df['METRIC_CD']=='B1S000700', pre_sub_weekly_df['P'], 0)
pre_sub_weekly_df['PRE_RPT_SB_T'] = np.where(pre_sub_weekly_df['METRIC_CD']=='TB1S000700', pre_sub_weekly_df['P'], 0)
pre_sub_weekly_df['PRE_RPT_SB_D'] = np.where(pre_sub_weekly_df['METRIC_CD']=='DB1S000700', pre_sub_weekly_df['P'], 0)

pre_sub_weekly_df = pre_sub_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max'
                                                        , 'PRE_SB':'sum', 'PRE_SB_T':'sum', 'PRE_SB_D':'sum'
                                                        , 'PRE_RPT_SB':'sum', 'PRE_RPT_SB_T':'sum', 'PRE_RPT_SB_D':'sum'})
pre_sub_weekly_df = pre_sub_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
pre_sub_weekly_df = pre_sub_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF'
                               , 'PRE_SB', 'PRE_SB_T', 'PRE_SB_D'
                               , 'PRE_RPT_SB', 'PRE_RPT_SB_T', 'PRE_RPT_SB_D']]

pre_sub_weekly_df_display = pre_sub_weekly_df.copy()
pre_sub_weekly_df_display['ACTUAL_AS_OF'] = pre_sub_weekly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = pre_sub_weekly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    pre_sub_weekly_df_display[col] = pre_sub_weekly_df_display[col].apply(lambda x: format(x, ',.0f'))
pre_sub_weekly_df_display

,TM_KEY_WK,PPN_TM,ACTUAL_AS_OF,PRE_SB,PRE_SB_T,PRE_SB_D,PRE_RPT_SB,PRE_RPT_SB_T,PRE_RPT_SB_D
0,2025018,2025-06-11 01:34:02,20250504,0,"23,741,610",0,0,"20,292,540",0
1,2025019,2025-06-11 01:34:02,20250511,0,"23,755,801",0,0,"20,255,191",0
2,2025020,2025-06-11 01:34:02,20250518,0,"23,835,575",0,0,"20,228,203",0
3,2025021,2025-06-11 01:34:02,20250525,0,"23,749,838",0,0,"20,149,647",0
4,2025022,2025-06-11 01:34:02,20250601,0,"23,717,576",0,0,"20,099,208",0
5,2025023,2025-06-11 01:34:02,20250608,0,"27,711,932",0,0,"19,830,123",0
6,2025024,2025-06-11 01:34:02,20250609,0,0,0,0,"23,803,763",0


In [9]:
''' Prepaid : SubBase Daily '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
    'B1S000702' #Prepaid SubBase
    , 'TB1S000702' #Prepaid SubBase : TMH
    , 'DB1S000702' #Prepaid SubBase : DTAC
	, 'B1S000700' #Prepaid Reported SubBase
	, 'TB1S000700' #Prepaid Reported SubBase : TMH
	, 'DB1S000700' #Prepaid Reported SubBase : DTAC
    ]

pre_sub_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth]
pre_sub_daily_df = pre_sub_daily_df.loc[pre_sub_daily_df['METRIC_CD'].isin(v_metric_list)]

pre_sub_daily_df['PRE_SB'] = np.where(pre_sub_daily_df['METRIC_CD']=='B1S000702', pre_sub_daily_df['P'], 0)
pre_sub_daily_df['PRE_SB_T'] = np.where(pre_sub_daily_df['METRIC_CD']=='TB1S000702', pre_sub_daily_df['P'], 0)
pre_sub_daily_df['PRE_SB_D'] = np.where(pre_sub_daily_df['METRIC_CD']=='DB1S000702', pre_sub_daily_df['P'], 0)

pre_sub_daily_df['PRE_RPT_SB'] = np.where(pre_sub_daily_df['METRIC_CD']=='B1S000700', pre_sub_daily_df['P'], 0)
pre_sub_daily_df['PRE_RPT_SB_T'] = np.where(pre_sub_daily_df['METRIC_CD']=='TB1S000700', pre_sub_daily_df['P'], 0)
pre_sub_daily_df['PRE_RPT_SB_D'] = np.where(pre_sub_daily_df['METRIC_CD']=='DB1S000700', pre_sub_daily_df['P'], 0)

pre_sub_daily_df = pre_sub_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max'
                                                        , 'PRE_SB':'sum', 'PRE_SB_T':'sum', 'PRE_SB_D':'sum'
                                                        , 'PRE_RPT_SB':'sum', 'PRE_RPT_SB_T':'sum', 'PRE_RPT_SB_D':'sum'})
pre_sub_daily_df = pre_sub_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
pre_sub_daily_df = pre_sub_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM'
                               , 'PRE_SB', 'PRE_SB_T', 'PRE_SB_D'
                               , 'PRE_RPT_SB', 'PRE_RPT_SB_T', 'PRE_RPT_SB_D']]

pre_sub_daily_df_display = pre_sub_daily_df.copy()
mod_col_list = pre_sub_daily_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    pre_sub_daily_df_display[col] = pre_sub_daily_df_display[col].apply(lambda x: format(x, ',.0f'))
pre_sub_daily_df_display

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,PRE_SB,PRE_SB_T,PRE_SB_D,PRE_RPT_SB,PRE_RPT_SB_T,PRE_RPT_SB_D
0,202506,20250610,2025-06-11 01:34:02,0,0,0,0,0,0
1,202506,20250609,2025-06-11 01:34:02,0,0,0,0,"23,803,763",0
2,202506,20250608,2025-06-11 01:34:02,0,"27,711,932",0,0,"19,830,123",0
3,202506,20250607,2025-06-11 01:34:02,0,"23,106,288",0,0,"19,860,221",0
4,202506,20250606,2025-06-11 01:34:02,0,"23,199,043",0,0,"19,888,701",0
5,202506,20250605,2025-06-11 01:34:02,0,"23,297,304",0,0,"19,924,299",0
6,202506,20250604,2025-06-11 01:34:02,0,"23,400,385",0,0,"19,969,790",0
7,202506,20250603,2025-06-11 01:34:02,0,"23,557,014",0,0,"20,032,338",0
8,202506,20250602,2025-06-11 01:34:02,0,"23,711,428",0,0,"20,082,815",0
9,202506,20250601,2025-06-11 01:34:02,0,"23,717,576",0,0,"20,099,208",0


## Postpaid B2C by Period

In [10]:
''' Postpaid B2C : SubBase Yearly '''

v_metric_list = [
	'B2S010602' #Postpaid SubBase B2C
    , 'TB2S010602' #Postpaid SubBase B2C : TMH
	, 'DB2S010602' #Postpaid SubBase B2C : DTAC

	, 'B2S010601' #Postpaid 60DPD B2C
    , 'TB2S010601' #Postpaid 60DPD B2C : TMH
	, 'DB2S010601' #Postpaid 60DPD B2C : DTAC

	, 'B2S010603' #Postpaid Gain/Loss 60DPD B2C
    , 'TB2S010603' #Postpaid Gain/Loss 60DPD B2C : TMH
	, 'DB2S010603' #Postpaid Gain/Loss 60DPD B2C : DTAC

	, 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB2S010600' #Postpaid Reported SubBase B2C : TMH
	, 'DB2S010600' #Postpaid Reported SubBase B2C : DTAC
    ]

post_b2c_sub_yearly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_YR', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_YR
post_b2c_sub_yearly_df = post_b2c_sub_yearly_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']]
post_b2c_sub_yearly_df = post_b2c_sub_yearly_df.loc[post_b2c_sub_yearly_df['METRIC_CD'].isin(v_metric_list)]

post_b2c_sub_yearly_df['POST_SB_B2C'] = np.where(post_b2c_sub_yearly_df['METRIC_CD']=='B2S010602', post_b2c_sub_yearly_df['P'], 0)
post_b2c_sub_yearly_df['POST_SB_B2C_T'] = np.where(post_b2c_sub_yearly_df['METRIC_CD']=='TB2S010602', post_b2c_sub_yearly_df['P'], 0)
post_b2c_sub_yearly_df['POST_SB_B2C_D'] = np.where(post_b2c_sub_yearly_df['METRIC_CD']=='DB2S010602', post_b2c_sub_yearly_df['P'], 0)

post_b2c_sub_yearly_df['POST_60DPD_B2C'] = np.where(post_b2c_sub_yearly_df['METRIC_CD']=='B2S010601', post_b2c_sub_yearly_df['P'], 0)
post_b2c_sub_yearly_df['POST_60DPD_B2C_T'] = np.where(post_b2c_sub_yearly_df['METRIC_CD']=='TB2S010601', post_b2c_sub_yearly_df['P'], 0)
post_b2c_sub_yearly_df['POST_60DPD_B2C_D'] = np.where(post_b2c_sub_yearly_df['METRIC_CD']=='DB2S010601', post_b2c_sub_yearly_df['P'], 0)

# post_b2c_sub_yearly_df['POST_GL_60DPD_B2C'] = np.where(post_b2c_sub_yearly_df['METRIC_CD']=='B2S010603', post_b2c_sub_yearly_df['P'], 0)
# post_b2c_sub_yearly_df['POST_GL_60DPD_B2C_T'] = np.where(post_b2c_sub_yearly_df['METRIC_CD']=='TB2S010603', post_b2c_sub_yearly_df['P'], 0)
# post_b2c_sub_yearly_df['POST_GL_60DPD_B2C_D'] = np.where(post_b2c_sub_yearly_df['METRIC_CD']=='DB2S010603', post_b2c_sub_yearly_df['P'], 0)

post_b2c_sub_yearly_df['POST_RPT_SB_B2C'] = np.where(post_b2c_sub_yearly_df['METRIC_CD']=='B2S010600', post_b2c_sub_yearly_df['P'], 0)
post_b2c_sub_yearly_df['POST_RPT_SB_B2C_T'] = np.where(post_b2c_sub_yearly_df['METRIC_CD']=='TB2S010600', post_b2c_sub_yearly_df['P'], 0)
post_b2c_sub_yearly_df['POST_RPT_SB_B2C_D'] = np.where(post_b2c_sub_yearly_df['METRIC_CD']=='DB2S010600', post_b2c_sub_yearly_df['P'], 0)

post_b2c_sub_yearly_df = post_b2c_sub_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max'
                                                        , 'POST_SB_B2C':'sum', 'POST_SB_B2C_T':'sum', 'POST_SB_B2C_D':'sum'
                                                        , 'POST_60DPD_B2C':'sum', 'POST_60DPD_B2C_T':'sum', 'POST_60DPD_B2C_D':'sum'
                                                        # , 'POST_GL_60DPD_B2C':'sum', 'POST_GL_60DPD_B2C_T':'sum', 'POST_GL_60DPD_B2C_D':'sum'
                                                        , 'POST_RPT_SB_B2C':'sum', 'POST_RPT_SB_B2C_T':'sum', 'POST_RPT_SB_B2C_D':'sum'})
post_b2c_sub_yearly_df = post_b2c_sub_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
post_b2c_sub_yearly_df = post_b2c_sub_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF'
                               , 'POST_SB_B2C', 'POST_SB_B2C_T', 'POST_SB_B2C_D'
                               , 'POST_60DPD_B2C', 'POST_60DPD_B2C_T', 'POST_60DPD_B2C_D'
                            #    , 'POST_GL_60DPD_B2C', 'POST_GL_60DPD_B2C_T', 'POST_GL_60DPD_B2C_D'
                               , 'POST_RPT_SB_B2C', 'POST_RPT_SB_B2C_T', 'POST_RPT_SB_B2C_D']]

post_b2c_sub_yearly_df_display = post_b2c_sub_yearly_df.copy()
post_b2c_sub_yearly_df_display['ACTUAL_AS_OF'] = post_b2c_sub_yearly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = post_b2c_sub_yearly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    post_b2c_sub_yearly_df_display[col] = post_b2c_sub_yearly_df_display[col].apply(lambda x: format(x, ',.0f'))
post_b2c_sub_yearly_df_display

,TM_KEY_YR,PPN_TM,ACTUAL_AS_OF,POST_SB_B2C,POST_SB_B2C_T,POST_SB_B2C_D,POST_60DPD_B2C,POST_60DPD_B2C_T,POST_60DPD_B2C_D,POST_RPT_SB_B2C,POST_RPT_SB_B2C_T,POST_RPT_SB_B2C_D
0,2024,2025-06-11 01:34:02,20241231,"14,287,442","9,102,725","5,184,717",0,"80,142","128,744","14,158,698","9,102,725","5,055,973"
1,2025,2025-06-11 01:34:02,20250609,"14,278,770","9,192,130","5,086,120",0,"89,461","122,681","14,064,757","9,102,669","4,962,088"


In [11]:
''' Postpaid B2C : SubBase Quarterly '''

v_metric_list = [
	'B2S010602' #Postpaid SubBase B2C
    , 'TB2S010602' #Postpaid SubBase B2C : TMH
	, 'DB2S010602' #Postpaid SubBase B2C : DTAC

	, 'B2S010601' #Postpaid 60DPD B2C
    , 'TB2S010601' #Postpaid 60DPD B2C : TMH
	, 'DB2S010601' #Postpaid 60DPD B2C : DTAC

	, 'B2S010603' #Postpaid Gain/Loss 60DPD B2C
    , 'TB2S010603' #Postpaid Gain/Loss 60DPD B2C : TMH
	, 'DB2S010603' #Postpaid Gain/Loss 60DPD B2C : DTAC

	, 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB2S010600' #Postpaid Reported SubBase B2C : TMH
	, 'DB2S010600' #Postpaid Reported SubBase B2C : DTAC
    ]

post_b2c_sub_quarterly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_QTR', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_QTR
post_b2c_sub_quarterly_df = post_b2c_sub_quarterly_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']]
post_b2c_sub_quarterly_df = post_b2c_sub_quarterly_df.loc[post_b2c_sub_quarterly_df['METRIC_CD'].isin(v_metric_list)]

post_b2c_sub_quarterly_df['POST_SB_B2C'] = np.where(post_b2c_sub_quarterly_df['METRIC_CD']=='B2S010602', post_b2c_sub_quarterly_df['P'], 0)
post_b2c_sub_quarterly_df['POST_SB_B2C_T'] = np.where(post_b2c_sub_quarterly_df['METRIC_CD']=='TB2S010602', post_b2c_sub_quarterly_df['P'], 0)
post_b2c_sub_quarterly_df['POST_SB_B2C_D'] = np.where(post_b2c_sub_quarterly_df['METRIC_CD']=='DB2S010602', post_b2c_sub_quarterly_df['P'], 0)

post_b2c_sub_quarterly_df['POST_60DPD_B2C'] = np.where(post_b2c_sub_quarterly_df['METRIC_CD']=='B2S010601', post_b2c_sub_quarterly_df['P'], 0)
post_b2c_sub_quarterly_df['POST_60DPD_B2C_T'] = np.where(post_b2c_sub_quarterly_df['METRIC_CD']=='TB2S010601', post_b2c_sub_quarterly_df['P'], 0)
post_b2c_sub_quarterly_df['POST_60DPD_B2C_D'] = np.where(post_b2c_sub_quarterly_df['METRIC_CD']=='DB2S010601', post_b2c_sub_quarterly_df['P'], 0)

# post_b2c_sub_quarterly_df['POST_GL_60DPD_B2C'] = np.where(post_b2c_sub_quarterly_df['METRIC_CD']=='B2S010603', post_b2c_sub_quarterly_df['P'], 0)
# post_b2c_sub_quarterly_df['POST_GL_60DPD_B2C_T'] = np.where(post_b2c_sub_quarterly_df['METRIC_CD']=='TB2S010603', post_b2c_sub_quarterly_df['P'], 0)
# post_b2c_sub_quarterly_df['POST_GL_60DPD_B2C_D'] = np.where(post_b2c_sub_quarterly_df['METRIC_CD']=='DB2S010603', post_b2c_sub_quarterly_df['P'], 0)

post_b2c_sub_quarterly_df['POST_RPT_SB_B2C'] = np.where(post_b2c_sub_quarterly_df['METRIC_CD']=='B2S010600', post_b2c_sub_quarterly_df['P'], 0)
post_b2c_sub_quarterly_df['POST_RPT_SB_B2C_T'] = np.where(post_b2c_sub_quarterly_df['METRIC_CD']=='TB2S010600', post_b2c_sub_quarterly_df['P'], 0)
post_b2c_sub_quarterly_df['POST_RPT_SB_B2C_D'] = np.where(post_b2c_sub_quarterly_df['METRIC_CD']=='DB2S010600', post_b2c_sub_quarterly_df['P'], 0)

post_b2c_sub_quarterly_df = post_b2c_sub_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max'
                                                        , 'POST_SB_B2C':'sum', 'POST_SB_B2C_T':'sum', 'POST_SB_B2C_D':'sum'
                                                        , 'POST_60DPD_B2C':'sum', 'POST_60DPD_B2C_T':'sum', 'POST_60DPD_B2C_D':'sum'
                                                        # , 'POST_GL_60DPD_B2C':'sum', 'POST_GL_60DPD_B2C_T':'sum', 'POST_GL_60DPD_B2C_D':'sum'
                                                        , 'POST_RPT_SB_B2C':'sum', 'POST_RPT_SB_B2C_T':'sum', 'POST_RPT_SB_B2C_D':'sum'})
post_b2c_sub_quarterly_df = post_b2c_sub_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
post_b2c_sub_quarterly_df = post_b2c_sub_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF'
                               , 'POST_SB_B2C', 'POST_SB_B2C_T', 'POST_SB_B2C_D'
                               , 'POST_60DPD_B2C', 'POST_60DPD_B2C_T', 'POST_60DPD_B2C_D'
                            #    , 'POST_GL_60DPD_B2C', 'POST_GL_60DPD_B2C_T', 'POST_GL_60DPD_B2C_D'
                               , 'POST_RPT_SB_B2C', 'POST_RPT_SB_B2C_T', 'POST_RPT_SB_B2C_D']]

post_b2c_sub_quarterly_df_display = post_b2c_sub_quarterly_df.copy()
post_b2c_sub_quarterly_df_display['ACTUAL_AS_OF'] = post_b2c_sub_quarterly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = post_b2c_sub_quarterly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    post_b2c_sub_quarterly_df_display[col] = post_b2c_sub_quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))
post_b2c_sub_quarterly_df_display

,TM_KEY_QTR,PPN_TM,ACTUAL_AS_OF,POST_SB_B2C,POST_SB_B2C_T,POST_SB_B2C_D,POST_60DPD_B2C,POST_60DPD_B2C_T,POST_60DPD_B2C_D,POST_RPT_SB_B2C,POST_RPT_SB_B2C_T,POST_RPT_SB_B2C_D
0,20241,2025-06-11 01:34:02,20240331,0,"8,994,853",0,0,"79,598",0,0,"8,994,853",0
1,20242,2025-06-11 01:34:02,20240630,"14,391,657","9,042,184","5,349,473",0,"81,727","165,490","14,226,167","9,042,184","5,183,983"
2,20243,2025-06-11 01:34:02,20240930,"14,338,027","9,068,470","5,269,557",0,"85,778","151,412","14,186,615","9,068,470","5,118,145"
3,20244,2025-06-11 01:34:02,20241231,"14,287,442","9,102,725","5,184,717",0,"80,142","128,744","14,158,698","9,102,725","5,055,973"
4,20251,2025-06-11 01:34:02,20250331,"14,288,650","9,170,605","5,118,045",0,"83,785","120,053","14,168,597","9,170,605","4,997,992"
5,20252,2025-06-11 01:34:02,20250609,"14,278,770","9,192,130","5,086,120",0,"89,461","122,681","14,064,757","9,102,669","4,962,088"


In [12]:
''' Postpaid B2C : SubBase Monthly '''

# v_tm_key_yr = curr_yr
v_tm_key_yr = prev_yr

v_metric_list = [
	'B2S010602' #Postpaid SubBase B2C
    , 'TB2S010602' #Postpaid SubBase B2C : TMH
	, 'DB2S010602' #Postpaid SubBase B2C : DTAC

	, 'B2S010601' #Postpaid 60DPD B2C
    , 'TB2S010601' #Postpaid 60DPD B2C : TMH
	, 'DB2S010601' #Postpaid 60DPD B2C : DTAC

	, 'B2S010603' #Postpaid Gain/Loss 60DPD B2C
    , 'TB2S010603' #Postpaid Gain/Loss 60DPD B2C : TMH
	, 'DB2S010603' #Postpaid Gain/Loss 60DPD B2C : DTAC

	, 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB2S010600' #Postpaid Reported SubBase B2C : TMH
	, 'DB2S010600' #Postpaid Reported SubBase B2C : DTAC
    ]

post_b2c_sub_monthly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_MTH', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_MTH
post_b2c_sub_monthly_df = post_b2c_sub_monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_YR']>=v_tm_key_yr]
post_b2c_sub_monthly_df = post_b2c_sub_monthly_df.loc[post_b2c_sub_monthly_df['METRIC_CD'].isin(v_metric_list)]

post_b2c_sub_monthly_df['POST_SB_B2C'] = np.where(post_b2c_sub_monthly_df['METRIC_CD']=='B2S010602', post_b2c_sub_monthly_df['P'], 0)
post_b2c_sub_monthly_df['POST_SB_B2C_T'] = np.where(post_b2c_sub_monthly_df['METRIC_CD']=='TB2S010602', post_b2c_sub_monthly_df['P'], 0)
post_b2c_sub_monthly_df['POST_SB_B2C_D'] = np.where(post_b2c_sub_monthly_df['METRIC_CD']=='DB2S010602', post_b2c_sub_monthly_df['P'], 0)

post_b2c_sub_monthly_df['POST_60DPD_B2C'] = np.where(post_b2c_sub_monthly_df['METRIC_CD']=='B2S010601', post_b2c_sub_monthly_df['P'], 0)
post_b2c_sub_monthly_df['POST_60DPD_B2C_T'] = np.where(post_b2c_sub_monthly_df['METRIC_CD']=='TB2S010601', post_b2c_sub_monthly_df['P'], 0)
post_b2c_sub_monthly_df['POST_60DPD_B2C_D'] = np.where(post_b2c_sub_monthly_df['METRIC_CD']=='DB2S010601', post_b2c_sub_monthly_df['P'], 0)

# post_b2c_sub_monthly_df['POST_GL_60DPD_B2C'] = np.where(post_b2c_sub_monthly_df['METRIC_CD']=='B2S010603', post_b2c_sub_monthly_df['P'], 0)
# post_b2c_sub_monthly_df['POST_GL_60DPD_B2C_T'] = np.where(post_b2c_sub_monthly_df['METRIC_CD']=='TB2S010603', post_b2c_sub_monthly_df['P'], 0)
# post_b2c_sub_monthly_df['POST_GL_60DPD_B2C_D'] = np.where(post_b2c_sub_monthly_df['METRIC_CD']=='DB2S010603', post_b2c_sub_monthly_df['P'], 0)

post_b2c_sub_monthly_df['POST_RPT_SB_B2C'] = np.where(post_b2c_sub_monthly_df['METRIC_CD']=='B2S010600', post_b2c_sub_monthly_df['P'], 0)
post_b2c_sub_monthly_df['POST_RPT_SB_B2C_T'] = np.where(post_b2c_sub_monthly_df['METRIC_CD']=='TB2S010600', post_b2c_sub_monthly_df['P'], 0)
post_b2c_sub_monthly_df['POST_RPT_SB_B2C_D'] = np.where(post_b2c_sub_monthly_df['METRIC_CD']=='DB2S010600', post_b2c_sub_monthly_df['P'], 0)

post_b2c_sub_monthly_df = post_b2c_sub_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max'
                                                        , 'POST_SB_B2C':'sum', 'POST_SB_B2C_T':'sum', 'POST_SB_B2C_D':'sum'
                                                        , 'POST_60DPD_B2C':'sum', 'POST_60DPD_B2C_T':'sum', 'POST_60DPD_B2C_D':'sum'
                                                        # , 'POST_GL_60DPD_B2C':'sum', 'POST_GL_60DPD_B2C_T':'sum', 'POST_GL_60DPD_B2C_D':'sum'
                                                        , 'POST_RPT_SB_B2C':'sum', 'POST_RPT_SB_B2C_T':'sum', 'POST_RPT_SB_B2C_D':'sum'})
post_b2c_sub_monthly_df = post_b2c_sub_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
post_b2c_sub_monthly_df = post_b2c_sub_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF'
                               , 'POST_SB_B2C', 'POST_SB_B2C_T', 'POST_SB_B2C_D'
                               , 'POST_60DPD_B2C', 'POST_60DPD_B2C_T', 'POST_60DPD_B2C_D'
                            #    , 'POST_GL_60DPD_B2C', 'POST_GL_60DPD_B2C_T', 'POST_GL_60DPD_B2C_D'
                               , 'POST_RPT_SB_B2C', 'POST_RPT_SB_B2C_T', 'POST_RPT_SB_B2C_D']]

post_b2c_sub_monthly_df_display = post_b2c_sub_monthly_df.copy()
post_b2c_sub_monthly_df_display['ACTUAL_AS_OF'] = post_b2c_sub_monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = post_b2c_sub_monthly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    post_b2c_sub_monthly_df_display[col] = post_b2c_sub_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
post_b2c_sub_monthly_df_display

,TM_KEY_MTH,PPN_TM,ACTUAL_AS_OF,POST_SB_B2C,POST_SB_B2C_T,POST_SB_B2C_D,POST_60DPD_B2C,POST_60DPD_B2C_T,POST_60DPD_B2C_D,POST_RPT_SB_B2C,POST_RPT_SB_B2C_T,POST_RPT_SB_B2C_D
0,202401,2025-06-11 01:34:02,20240131,0,0,0,0,0,0,0,0,0
1,202402,2025-06-11 01:34:02,20240229,0,"8,998,482",0,0,"98,379",0,0,"8,998,482",0
2,202403,2025-06-11 01:34:02,20240331,0,"8,994,853",0,0,"79,598",0,0,"8,994,853",0
3,202404,2025-06-11 01:34:02,20240430,0,0,0,0,0,0,0,0,0
4,202405,2025-06-11 01:34:02,20240531,"14,395,841","9,023,324","5,372,517",0,"76,251","171,764",0,"9,023,324",0
5,202406,2025-06-11 01:34:02,20240630,"14,391,657","9,042,184","5,349,473",0,"81,727","165,490","14,226,167","9,042,184","5,183,983"
6,202407,2025-06-11 01:34:02,20240731,"14,372,506","9,052,569","5,319,937",0,"83,942","163,473","14,209,033","9,052,569","5,156,464"
7,202408,2025-06-11 01:34:02,20240831,"14,353,346","9,061,002","5,292,344",0,"86,053","162,915","14,190,431","9,061,002","5,129,429"
8,202409,2025-06-11 01:34:02,20240930,"14,338,027","9,068,470","5,269,557",0,"85,778","151,412","14,186,615","9,068,470","5,118,145"
9,202410,2025-06-11 01:34:02,20241031,"14,321,392","9,080,143","5,241,249",0,"85,630","141,986","14,179,406","9,080,143","5,099,263"


In [13]:
''' Postpaid B2C : SubBase Weekly '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
	'B2S010602' #Postpaid SubBase B2C
    , 'TB2S010602' #Postpaid SubBase B2C : TMH
	, 'DB2S010602' #Postpaid SubBase B2C : DTAC

	, 'B2S010601' #Postpaid 60DPD B2C
    , 'TB2S010601' #Postpaid 60DPD B2C : TMH
	, 'DB2S010601' #Postpaid 60DPD B2C : DTAC

	, 'B2S010603' #Postpaid Gain/Loss 60DPD B2C
    , 'TB2S010603' #Postpaid Gain/Loss 60DPD B2C : TMH
	, 'DB2S010603' #Postpaid Gain/Loss 60DPD B2C : DTAC

	, 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB2S010600' #Postpaid Reported SubBase B2C : TMH
	, 'DB2S010600' #Postpaid Reported SubBase B2C : DTAC
    ]

post_b2c_sub_weekly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_WK', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_WK
post_b2c_sub_weekly_df = post_b2c_sub_weekly_df[['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth]
post_b2c_sub_weekly_df = post_b2c_sub_weekly_df.loc[post_b2c_sub_weekly_df['METRIC_CD'].isin(v_metric_list)]

post_b2c_sub_weekly_df['POST_SB_B2C'] = np.where(post_b2c_sub_weekly_df['METRIC_CD']=='B2S010602', post_b2c_sub_weekly_df['P'], 0)
post_b2c_sub_weekly_df['POST_SB_B2C_T'] = np.where(post_b2c_sub_weekly_df['METRIC_CD']=='TB2S010602', post_b2c_sub_weekly_df['P'], 0)
post_b2c_sub_weekly_df['POST_SB_B2C_D'] = np.where(post_b2c_sub_weekly_df['METRIC_CD']=='DB2S010602', post_b2c_sub_weekly_df['P'], 0)

post_b2c_sub_weekly_df['POST_60DPD_B2C'] = np.where(post_b2c_sub_weekly_df['METRIC_CD']=='B2S010601', post_b2c_sub_weekly_df['P'], 0)
post_b2c_sub_weekly_df['POST_60DPD_B2C_T'] = np.where(post_b2c_sub_weekly_df['METRIC_CD']=='TB2S010601', post_b2c_sub_weekly_df['P'], 0)
post_b2c_sub_weekly_df['POST_60DPD_B2C_D'] = np.where(post_b2c_sub_weekly_df['METRIC_CD']=='DB2S010601', post_b2c_sub_weekly_df['P'], 0)

# post_b2c_sub_weekly_df['POST_GL_60DPD_B2C'] = np.where(post_b2c_sub_weekly_df['METRIC_CD']=='B2S010603', post_b2c_sub_weekly_df['P'], 0)
# post_b2c_sub_weekly_df['POST_GL_60DPD_B2C_T'] = np.where(post_b2c_sub_weekly_df['METRIC_CD']=='TB2S010603', post_b2c_sub_weekly_df['P'], 0)
# post_b2c_sub_weekly_df['POST_GL_60DPD_B2C_D'] = np.where(post_b2c_sub_weekly_df['METRIC_CD']=='DB2S010603', post_b2c_sub_weekly_df['P'], 0)

post_b2c_sub_weekly_df['POST_RPT_SB_B2C'] = np.where(post_b2c_sub_weekly_df['METRIC_CD']=='B2S010600', post_b2c_sub_weekly_df['P'], 0)
post_b2c_sub_weekly_df['POST_RPT_SB_B2C_T'] = np.where(post_b2c_sub_weekly_df['METRIC_CD']=='TB2S010600', post_b2c_sub_weekly_df['P'], 0)
post_b2c_sub_weekly_df['POST_RPT_SB_B2C_D'] = np.where(post_b2c_sub_weekly_df['METRIC_CD']=='DB2S010600', post_b2c_sub_weekly_df['P'], 0)

post_b2c_sub_weekly_df = post_b2c_sub_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max'
                                                        , 'POST_SB_B2C':'sum', 'POST_SB_B2C_T':'sum', 'POST_SB_B2C_D':'sum'
                                                        , 'POST_60DPD_B2C':'sum', 'POST_60DPD_B2C_T':'sum', 'POST_60DPD_B2C_D':'sum'
                                                        # , 'POST_GL_60DPD_B2C':'sum', 'POST_GL_60DPD_B2C_T':'sum', 'POST_GL_60DPD_B2C_D':'sum'
                                                        , 'POST_RPT_SB_B2C':'sum', 'POST_RPT_SB_B2C_T':'sum', 'POST_RPT_SB_B2C_D':'sum'})
post_b2c_sub_weekly_df = post_b2c_sub_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
post_b2c_sub_weekly_df = post_b2c_sub_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF'
                               , 'POST_SB_B2C', 'POST_SB_B2C_T', 'POST_SB_B2C_D'
                               , 'POST_60DPD_B2C', 'POST_60DPD_B2C_T', 'POST_60DPD_B2C_D'
                            #    , 'POST_GL_60DPD_B2C', 'POST_GL_60DPD_B2C_T', 'POST_GL_60DPD_B2C_D'
                               , 'POST_RPT_SB_B2C', 'POST_RPT_SB_B2C_T', 'POST_RPT_SB_B2C_D']]

post_b2c_sub_weekly_df_display = post_b2c_sub_weekly_df.copy()
post_b2c_sub_weekly_df_display['ACTUAL_AS_OF'] = post_b2c_sub_weekly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = post_b2c_sub_weekly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    post_b2c_sub_weekly_df_display[col] = post_b2c_sub_weekly_df_display[col].apply(lambda x: format(x, ',.0f'))
post_b2c_sub_weekly_df_display

,TM_KEY_WK,PPN_TM,ACTUAL_AS_OF,POST_SB_B2C,POST_SB_B2C_T,POST_SB_B2C_D,POST_60DPD_B2C,POST_60DPD_B2C_T,POST_60DPD_B2C_D,POST_RPT_SB_B2C,POST_RPT_SB_B2C_T,POST_RPT_SB_B2C_D
0,2025018,2025-06-11 01:34:02,20250504,"14,301,571","9,194,975","5,106,596",0,"84,668","119,511","14,097,392","9,110,307","4,987,085"
1,2025019,2025-06-11 01:34:02,20250511,"14,300,907","9,200,984","5,099,923",0,"85,091","118,610","14,182,297","9,200,984","4,981,313"
2,2025020,2025-06-11 01:34:02,20250518,"14,300,979","9,202,580","5,098,399",0,"96,885","123,717","14,177,262","9,202,580","4,974,682"
3,2025021,2025-06-11 01:34:02,20250525,"14,296,786","9,203,124","5,093,662",0,"100,664","128,032","14,068,090","9,102,460","4,965,630"
4,2025022,2025-06-11 01:34:02,20250601,"14,286,856","9,194,914","5,091,942",0,"88,412","124,725","14,162,131","9,194,914","4,967,217"
5,2025023,2025-06-11 01:34:02,20250608,"14,278,770","9,192,130","5,086,640",0,"89,461","124,552","14,064,757","9,102,669","4,962,088"
6,2025024,2025-06-11 01:34:02,20250609,0,0,"5,086,120",0,0,"122,681",0,0,0


In [14]:
''' Postpaid B2C : SubBase Daily '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
	'B2S010602' #Postpaid SubBase B2C
    , 'TB2S010602' #Postpaid SubBase B2C : TMH
	, 'DB2S010602' #Postpaid SubBase B2C : DTAC

	, 'B2S010601' #Postpaid 60DPD B2C
    , 'TB2S010601' #Postpaid 60DPD B2C : TMH
	, 'DB2S010601' #Postpaid 60DPD B2C : DTAC

	, 'B2S010603' #Postpaid Gain/Loss 60DPD B2C
    , 'TB2S010603' #Postpaid Gain/Loss 60DPD B2C : TMH
	, 'DB2S010603' #Postpaid Gain/Loss 60DPD B2C : DTAC

	, 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB2S010600' #Postpaid Reported SubBase B2C : TMH
	, 'DB2S010600' #Postpaid Reported SubBase B2C : DTAC
    ]

post_b2c_sub_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth]
post_b2c_sub_daily_df = post_b2c_sub_daily_df.loc[post_b2c_sub_daily_df['METRIC_CD'].isin(v_metric_list)]

post_b2c_sub_daily_df['POST_SB_B2C'] = np.where(post_b2c_sub_daily_df['METRIC_CD']=='B2S010602', post_b2c_sub_daily_df['P'], 0)
post_b2c_sub_daily_df['POST_SB_B2C_T'] = np.where(post_b2c_sub_daily_df['METRIC_CD']=='TB2S010602', post_b2c_sub_daily_df['P'], 0)
post_b2c_sub_daily_df['POST_SB_B2C_D'] = np.where(post_b2c_sub_daily_df['METRIC_CD']=='DB2S010602', post_b2c_sub_daily_df['P'], 0)

post_b2c_sub_daily_df['POST_60DPD_B2C'] = np.where(post_b2c_sub_daily_df['METRIC_CD']=='B2S010601', post_b2c_sub_daily_df['P'], 0)
post_b2c_sub_daily_df['POST_60DPD_B2C_T'] = np.where(post_b2c_sub_daily_df['METRIC_CD']=='TB2S010601', post_b2c_sub_daily_df['P'], 0)
post_b2c_sub_daily_df['POST_60DPD_B2C_D'] = np.where(post_b2c_sub_daily_df['METRIC_CD']=='DB2S010601', post_b2c_sub_daily_df['P'], 0)

# post_b2c_sub_daily_df['POST_GL_60DPD_B2C'] = np.where(post_b2c_sub_daily_df['METRIC_CD']=='B2S010603', post_b2c_sub_daily_df['P'], 0)
# post_b2c_sub_daily_df['POST_GL_60DPD_B2C_T'] = np.where(post_b2c_sub_daily_df['METRIC_CD']=='TB2S010603', post_b2c_sub_daily_df['P'], 0)
# post_b2c_sub_daily_df['POST_GL_60DPD_B2C_D'] = np.where(post_b2c_sub_daily_df['METRIC_CD']=='DB2S010603', post_b2c_sub_daily_df['P'], 0)

post_b2c_sub_daily_df['POST_RPT_SB_B2C'] = np.where(post_b2c_sub_daily_df['METRIC_CD']=='B2S010600', post_b2c_sub_daily_df['P'], 0)
post_b2c_sub_daily_df['POST_RPT_SB_B2C_T'] = np.where(post_b2c_sub_daily_df['METRIC_CD']=='TB2S010600', post_b2c_sub_daily_df['P'], 0)
post_b2c_sub_daily_df['POST_RPT_SB_B2C_D'] = np.where(post_b2c_sub_daily_df['METRIC_CD']=='DB2S010600', post_b2c_sub_daily_df['P'], 0)

post_b2c_sub_daily_df = post_b2c_sub_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max'
                                                        , 'POST_SB_B2C':'sum', 'POST_SB_B2C_T':'sum', 'POST_SB_B2C_D':'sum'
                                                        , 'POST_60DPD_B2C':'sum', 'POST_60DPD_B2C_T':'sum', 'POST_60DPD_B2C_D':'sum'
                                                        # , 'POST_GL_60DPD_B2C':'sum', 'POST_GL_60DPD_B2C_T':'sum', 'POST_GL_60DPD_B2C_D':'sum'
                                                        , 'POST_RPT_SB_B2C':'sum', 'POST_RPT_SB_B2C_T':'sum', 'POST_RPT_SB_B2C_D':'sum'})
post_b2c_sub_daily_df = post_b2c_sub_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
post_b2c_sub_daily_df = post_b2c_sub_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM'
                               , 'POST_SB_B2C', 'POST_SB_B2C_T', 'POST_SB_B2C_D'
                               , 'POST_60DPD_B2C', 'POST_60DPD_B2C_T', 'POST_60DPD_B2C_D'
                            #    , 'POST_GL_60DPD_B2C', 'POST_GL_60DPD_B2C_T', 'POST_GL_60DPD_B2C_D'
                               , 'POST_RPT_SB_B2C', 'POST_RPT_SB_B2C_T', 'POST_RPT_SB_B2C_D']]

post_b2c_sub_daily_df_display = post_b2c_sub_daily_df.copy()
mod_col_list = post_b2c_sub_daily_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    post_b2c_sub_daily_df_display[col] = post_b2c_sub_daily_df_display[col].apply(lambda x: format(x, ',.0f'))
post_b2c_sub_daily_df_display

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,POST_SB_B2C,POST_SB_B2C_T,POST_SB_B2C_D,POST_60DPD_B2C,POST_60DPD_B2C_T,POST_60DPD_B2C_D,POST_RPT_SB_B2C,POST_RPT_SB_B2C_T,POST_RPT_SB_B2C_D
0,202506,20250610,2025-06-11 01:34:02,0,0,0,0,0,0,0,0,0
1,202506,20250609,2025-06-11 01:34:02,0,0,"5,086,120",0,0,"122,681",0,0,0
2,202506,20250608,2025-06-11 01:34:02,"14,278,770","9,192,130","5,086,640",0,"89,461","124,552","14,064,757","9,102,669","4,962,088"
3,202506,20250607,2025-06-11 01:34:02,"14,279,395","9,192,281","5,087,114",0,"92,446","125,904","14,061,045","9,099,835","4,961,210"
4,202506,20250606,2025-06-11 01:34:02,"14,278,463","9,192,157","5,086,306",0,"94,787","126,587","14,057,089","9,097,370","4,959,719"
5,202506,20250605,2025-06-11 01:34:02,"14,281,294","9,193,029","5,088,265",0,"88,485","123,467","14,069,342","9,104,544","4,964,798"
6,202506,20250604,2025-06-11 01:34:02,"14,286,832","9,196,579","5,090,253",0,"91,944","124,430","14,070,458","9,104,635","4,965,823"
7,202506,20250603,2025-06-11 01:34:02,"14,286,859","9,196,790","5,090,069",0,"94,736","125,751","14,066,372","9,102,054","4,964,318"
8,202506,20250602,2025-06-11 01:34:02,"14,283,569","9,194,668","5,088,901",0,"85,937","126,518","14,071,114","9,108,731","4,962,383"
9,202506,20250601,2025-06-11 01:34:02,"14,286,856","9,194,914","5,091,942",0,"88,412","124,725","14,162,131","9,194,914","4,967,217"


## FTTx by Period

In [15]:
''' FTTx : SubBase Yearly '''

v_metric_list = [
    'TB3S020604' #FTTx SubBase
	, 'TB3S020603' #FTTx 60DPD
	, 'TB3S020606' #FTTx Gain/Loss 60DPD
	, 'TB3S000600' #FTTx Reported SubBase
    ]

fttx_sub_yearly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_YR', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_YR
fttx_sub_yearly_df = fttx_sub_yearly_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']]
fttx_sub_yearly_df = fttx_sub_yearly_df.loc[fttx_sub_yearly_df['METRIC_CD'].isin(v_metric_list)]

fttx_sub_yearly_df['FTTX_SB'] = np.where(fttx_sub_yearly_df['METRIC_CD']=='TB3S020604', fttx_sub_yearly_df['P'], 0)
fttx_sub_yearly_df['FTTX_60DPD'] = np.where(fttx_sub_yearly_df['METRIC_CD']=='TB3S020603', fttx_sub_yearly_df['P'], 0)
fttx_sub_yearly_df['FTTX_GL_60DPD'] = np.where(fttx_sub_yearly_df['METRIC_CD']=='TB3S020606', fttx_sub_yearly_df['P'], 0)
fttx_sub_yearly_df['FTTX_RPT_SB'] = np.where(fttx_sub_yearly_df['METRIC_CD']=='TB3S000600', fttx_sub_yearly_df['P'], 0)

fttx_sub_yearly_df = fttx_sub_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max'
                                                        , 'FTTX_SB':'sum', 'FTTX_60DPD':'sum', 'FTTX_GL_60DPD':'sum', 'FTTX_RPT_SB':'sum'})
fttx_sub_yearly_df = fttx_sub_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
fttx_sub_yearly_df = fttx_sub_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF'
                               , 'FTTX_SB', 'FTTX_60DPD', 'FTTX_GL_60DPD', 'FTTX_RPT_SB']]

fttx_sub_yearly_df_display = fttx_sub_yearly_df.copy()
fttx_sub_yearly_df_display['ACTUAL_AS_OF'] = fttx_sub_yearly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = fttx_sub_yearly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    fttx_sub_yearly_df_display[col] = fttx_sub_yearly_df_display[col].apply(lambda x: format(x, ',.0f'))
fttx_sub_yearly_df_display

,TM_KEY_YR,PPN_TM,ACTUAL_AS_OF,FTTX_SB,FTTX_60DPD,FTTX_GL_60DPD,FTTX_RPT_SB
0,2024,2025-06-11 01:34:02,20241231,"3,270,700","115,811","6,834","3,154,889"
1,2025,2025-06-11 01:34:02,20250608,"3,301,794","99,072",-974,"3,202,722"


In [16]:
''' FTTx : SubBase Quarterly '''

v_metric_list = [
    'TB3S020604' #FTTx SubBase
	, 'TB3S020603' #FTTx 60DPD
	, 'TB3S020606' #FTTx Gain/Loss 60DPD
	, 'TB3S000600' #FTTx Reported SubBase
    ]

fttx_sub_quarterly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_QTR', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_QTR
fttx_sub_quarterly_df = fttx_sub_quarterly_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']]
fttx_sub_quarterly_df = fttx_sub_quarterly_df.loc[fttx_sub_quarterly_df['METRIC_CD'].isin(v_metric_list)]

fttx_sub_quarterly_df['FTTX_SB'] = np.where(fttx_sub_quarterly_df['METRIC_CD']=='TB3S020604', fttx_sub_quarterly_df['P'], 0)
fttx_sub_quarterly_df['FTTX_60DPD'] = np.where(fttx_sub_quarterly_df['METRIC_CD']=='TB3S020603', fttx_sub_quarterly_df['P'], 0)
fttx_sub_quarterly_df['FTTX_GL_60DPD'] = np.where(fttx_sub_quarterly_df['METRIC_CD']=='TB3S020606', fttx_sub_quarterly_df['P'], 0)
fttx_sub_quarterly_df['FTTX_RPT_SB'] = np.where(fttx_sub_quarterly_df['METRIC_CD']=='TB3S000600', fttx_sub_quarterly_df['P'], 0)

fttx_sub_quarterly_df = fttx_sub_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max'
                                                        , 'FTTX_SB':'sum', 'FTTX_60DPD':'sum', 'FTTX_GL_60DPD':'sum', 'FTTX_RPT_SB':'sum'})
fttx_sub_quarterly_df = fttx_sub_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
fttx_sub_quarterly_df = fttx_sub_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF'
                               , 'FTTX_SB', 'FTTX_60DPD', 'FTTX_GL_60DPD', 'FTTX_RPT_SB']]

fttx_sub_quarterly_df_display = fttx_sub_quarterly_df.copy()
fttx_sub_quarterly_df_display['ACTUAL_AS_OF'] = fttx_sub_quarterly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = fttx_sub_quarterly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    fttx_sub_quarterly_df_display[col] = fttx_sub_quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))
fttx_sub_quarterly_df_display

,TM_KEY_QTR,PPN_TM,ACTUAL_AS_OF,FTTX_SB,FTTX_60DPD,FTTX_GL_60DPD,FTTX_RPT_SB
0,20241,2025-06-11 01:34:02,20240331,"3,200,919","110,632",649,"3,090,287"
1,20242,2025-06-11 01:34:02,20240630,"3,219,954","112,870",636,"3,107,084"
2,20243,2025-06-11 01:34:02,20240930,"3,246,868","119,564","2,863","3,127,304"
3,20244,2025-06-11 01:34:02,20241231,"3,270,700","115,811","6,834","3,154,889"
4,20251,2025-06-11 01:34:02,20250331,"3,305,177","117,460","2,510","3,187,717"
5,20252,2025-06-11 01:34:02,20250608,"3,301,794","99,072",-974,"3,202,722"


In [17]:
''' FTTx : SubBase Monthly '''

# v_tm_key_yr = curr_yr
v_tm_key_yr = prev_yr

v_metric_list = [
    'TB3S020604' #FTTx SubBase
	, 'TB3S020603' #FTTx 60DPD
	, 'TB3S020606' #FTTx Gain/Loss 60DPD
	, 'TB3S000600' #FTTx Reported SubBase
    ]

fttx_sub_monthly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_MTH', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_MTH
fttx_sub_monthly_df = fttx_sub_monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_YR']>=v_tm_key_yr]
fttx_sub_monthly_df = fttx_sub_monthly_df.loc[fttx_sub_monthly_df['METRIC_CD'].isin(v_metric_list)]

fttx_sub_monthly_df['FTTX_SB'] = np.where(fttx_sub_monthly_df['METRIC_CD']=='TB3S020604', fttx_sub_monthly_df['P'], 0)
fttx_sub_monthly_df['FTTX_60DPD'] = np.where(fttx_sub_monthly_df['METRIC_CD']=='TB3S020603', fttx_sub_monthly_df['P'], 0)
fttx_sub_monthly_df['FTTX_GL_60DPD'] = np.where(fttx_sub_monthly_df['METRIC_CD']=='TB3S020606', fttx_sub_monthly_df['P'], 0)
fttx_sub_monthly_df['FTTX_RPT_SB'] = np.where(fttx_sub_monthly_df['METRIC_CD']=='TB3S000600', fttx_sub_monthly_df['P'], 0)

fttx_sub_monthly_df = fttx_sub_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max'
                                                        , 'FTTX_SB':'sum', 'FTTX_60DPD':'sum', 'FTTX_GL_60DPD':'sum', 'FTTX_RPT_SB':'sum'})
fttx_sub_monthly_df = fttx_sub_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
fttx_sub_monthly_df = fttx_sub_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF'
                               , 'FTTX_SB', 'FTTX_60DPD', 'FTTX_GL_60DPD', 'FTTX_RPT_SB']]

fttx_sub_monthly_df_display = fttx_sub_monthly_df.copy()
fttx_sub_monthly_df_display['ACTUAL_AS_OF'] = fttx_sub_monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = fttx_sub_monthly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    fttx_sub_monthly_df_display[col] = fttx_sub_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
fttx_sub_monthly_df_display

,TM_KEY_MTH,PPN_TM,ACTUAL_AS_OF,FTTX_SB,FTTX_60DPD,FTTX_GL_60DPD,FTTX_RPT_SB
0,202402,2025-06-11 01:34:02,20240229,"3,196,974","112,709","1,759","3,084,265"
1,202403,2025-06-11 01:34:02,20240331,"3,200,919","110,632",649,"3,090,287"
2,202404,2025-06-11 01:34:02,20240430,"3,202,928","111,161","-1,855","3,091,767"
3,202405,2025-06-11 01:34:02,20240531,"3,209,626","111,112","2,194","3,098,514"
4,202406,2025-06-11 01:34:02,20240630,"3,219,954","112,870",636,"3,107,084"
5,202407,2025-06-11 01:34:02,20240731,"3,229,133","114,298",959,"3,114,835"
6,202408,2025-06-11 01:34:02,20240831,"3,239,197","117,168",985,"3,122,029"
7,202409,2025-06-11 01:34:02,20240930,"3,246,868","119,564","2,863","3,127,304"
8,202410,2025-06-11 01:34:02,20241031,"3,255,234","120,867","-1,414","3,134,367"
9,202411,2025-06-11 01:34:02,20241130,"3,262,940","124,095","1,240","3,138,845"


In [18]:
''' FTTx : SubBase Weekly '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
    'TB3S020604' #FTTx SubBase
	, 'TB3S020603' #FTTx 60DPD
	, 'TB3S020606' #FTTx Gain/Loss 60DPD
	, 'TB3S000600' #FTTx Reported SubBase
    ]

fttx_sub_weekly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_WK', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_WK
fttx_sub_weekly_df = fttx_sub_weekly_df[['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth]
fttx_sub_weekly_df = fttx_sub_weekly_df.loc[fttx_sub_weekly_df['METRIC_CD'].isin(v_metric_list)]

fttx_sub_weekly_df['FTTX_SB'] = np.where(fttx_sub_weekly_df['METRIC_CD']=='TB3S020604', fttx_sub_weekly_df['P'], 0)
fttx_sub_weekly_df['FTTX_60DPD'] = np.where(fttx_sub_weekly_df['METRIC_CD']=='TB3S020603', fttx_sub_weekly_df['P'], 0)
fttx_sub_weekly_df['FTTX_GL_60DPD'] = np.where(fttx_sub_weekly_df['METRIC_CD']=='TB3S020606', fttx_sub_weekly_df['P'], 0)
fttx_sub_weekly_df['FTTX_RPT_SB'] = np.where(fttx_sub_weekly_df['METRIC_CD']=='TB3S000600', fttx_sub_weekly_df['P'], 0)

fttx_sub_weekly_df = fttx_sub_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max'
                                                        , 'FTTX_SB':'sum', 'FTTX_60DPD':'sum', 'FTTX_GL_60DPD':'sum', 'FTTX_RPT_SB':'sum'})
fttx_sub_weekly_df = fttx_sub_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
fttx_sub_weekly_df = fttx_sub_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF'
                               , 'FTTX_SB', 'FTTX_60DPD', 'FTTX_GL_60DPD', 'FTTX_RPT_SB']]

fttx_sub_weekly_df_display = fttx_sub_weekly_df.copy()
fttx_sub_weekly_df_display['ACTUAL_AS_OF'] = fttx_sub_weekly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = fttx_sub_weekly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    fttx_sub_weekly_df_display[col] = fttx_sub_weekly_df_display[col].apply(lambda x: format(x, ',.0f'))
fttx_sub_weekly_df_display

,TM_KEY_WK,PPN_TM,ACTUAL_AS_OF,FTTX_SB,FTTX_60DPD,FTTX_GL_60DPD,FTTX_RPT_SB
0,2025018,2025-06-11 01:34:02,20250504,"3,318,455","129,506",694,"3,188,949"
1,2025019,2025-06-11 01:34:02,20250511,"3,323,804","131,424",0,"3,192,380"
2,2025020,2025-06-11 01:34:02,20250518,"3,313,557","121,619","-5,038","3,191,938"
3,2025021,2025-06-11 01:34:02,20250525,"3,294,137","100,767","-6,226","3,193,370"
4,2025022,2025-06-11 01:34:02,20250601,"3,295,710","90,238","-1,455","3,205,472"
5,2025023,2025-06-11 01:34:02,20250608,"3,301,794","99,072",-974,"3,202,722"


In [19]:
''' FTTx : SubBase Daily '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
    'TB3S020604' #FTTx SubBase
	, 'TB3S020603' #FTTx 60DPD
	, 'TB3S020606' #FTTx Gain/Loss 60DPD
	, 'TB3S000600' #FTTx Reported SubBase
    ]

fttx_sub_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth]
fttx_sub_daily_df = fttx_sub_daily_df.loc[fttx_sub_daily_df['METRIC_CD'].isin(v_metric_list)]

fttx_sub_daily_df['FTTX_SB'] = np.where(fttx_sub_daily_df['METRIC_CD']=='TB3S020604', fttx_sub_daily_df['P'], 0)
fttx_sub_daily_df['FTTX_60DPD'] = np.where(fttx_sub_daily_df['METRIC_CD']=='TB3S020603', fttx_sub_daily_df['P'], 0)
fttx_sub_daily_df['FTTX_GL_60DPD'] = np.where(fttx_sub_daily_df['METRIC_CD']=='TB3S020606', fttx_sub_daily_df['P'], 0)
fttx_sub_daily_df['FTTX_RPT_SB'] = np.where(fttx_sub_daily_df['METRIC_CD']=='TB3S000600', fttx_sub_daily_df['P'], 0)

fttx_sub_daily_df = fttx_sub_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max'
                                                        , 'FTTX_SB':'sum', 'FTTX_60DPD':'sum', 'FTTX_GL_60DPD':'sum', 'FTTX_RPT_SB':'sum'})
fttx_sub_daily_df = fttx_sub_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
fttx_sub_daily_df = fttx_sub_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM'
                               , 'FTTX_SB', 'FTTX_60DPD', 'FTTX_GL_60DPD', 'FTTX_RPT_SB']]

fttx_sub_daily_df_display = fttx_sub_daily_df.copy()
mod_col_list = fttx_sub_daily_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    fttx_sub_daily_df_display[col] = fttx_sub_daily_df_display[col].apply(lambda x: format(x, ',.0f'))
fttx_sub_daily_df_display

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,FTTX_SB,FTTX_60DPD,FTTX_GL_60DPD,FTTX_RPT_SB
0,202506,20250610,2025-06-11 01:34:02,0,0,0,0
1,202506,20250609,2025-06-11 01:34:02,0,0,0,0
2,202506,20250608,2025-06-11 01:34:02,"3,301,794","99,072",-974,"3,202,722"
3,202506,20250607,2025-06-11 01:34:02,"3,300,792","100,046","-1,046","3,200,746"
4,202506,20250606,2025-06-11 01:34:02,"3,299,580","101,092","3,476","3,198,488"
5,202506,20250605,2025-06-11 01:34:02,"3,300,191","97,616","-1,112","3,202,575"
6,202506,20250604,2025-06-11 01:34:02,"3,299,041","98,728","-1,017","3,200,313"
7,202506,20250603,2025-06-11 01:34:02,"3,297,765","99,745","9,507","3,198,020"
8,202506,20250602,2025-06-11 01:34:02,"3,296,754","90,238",0,"3,206,516"
9,202506,20250601,2025-06-11 01:34:02,"3,295,710","90,238","-1,455","3,205,472"


## Products Summary

### Prep Monthly Data

In [20]:
''' Monthly Summary '''

monthly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_MTH', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_MTH
monthly_df = monthly_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF']).agg({'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
monthly_df = monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
monthly_df['DIFF (C-P)'] = monthly_df['C'] - monthly_df['P']

monthly_df_display = monthly_df.copy()
monthly_df_display['ACTUAL_AS_OF'] = monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = monthly_df_display.iloc[:, 6:].columns.tolist()
for col in mod_col_list:
    monthly_df_display[col] = monthly_df_display[col].apply(lambda x: format(x, ',.0f'))

# monthly_df_display
# monthly_df_display.loc[monthly_df_display['TM_KEY_MTH']==202501]

In [24]:
''' Parameter '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth
# v_tm_key_mth = 202501

print(f'v_tm_key_mth: {v_tm_key_mth}')

v_tm_key_mth: 202506


### Prepaid

In [26]:
''' Prepaid SubBase '''

v_metric_list = [
    'B1S000702' #Prepaid SubBase
    , 'TB1S000702' #Prepaid SubBase : TMH
    , 'DB1S000702' #Prepaid SubBase : DTAC
    ]

prepaid_subbase_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
prepaid_subbase_df = prepaid_subbase_df.loc[prepaid_subbase_df['TM_KEY_MTH']>=v_tm_key_mth]
prepaid_subbase_df = prepaid_subbase_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
prepaid_subbase_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202506,Prepaid,TB1S000702,Prepaid SubBase : TMH,2025-06-11 01:34:02,20250608,0,"27,711,932","27,539,388","27,538,597","27,538,597","-27,711,932"


In [28]:
''' Prepaid Reported SubBase '''

v_metric_list = [
    'B1S000700' #Prepaid Reported SubBase
    , 'TB1S000700' #Prepaid Reported SubBase : TMH
    , 'DB1S000700' #Prepaid Reported SubBase : DTAC
    ]

prepaid_reported_subbase_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
prepaid_reported_subbase_df = prepaid_reported_subbase_df.loc[prepaid_reported_subbase_df['TM_KEY_MTH']>=v_tm_key_mth]
prepaid_reported_subbase_df = prepaid_reported_subbase_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
prepaid_reported_subbase_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202506,Prepaid,TB1S000700,Prepaid Reported SubBase : TMH,2025-06-11 01:34:02,20250609,0,"23,803,763","23,646,387","23,646,387","23,646,387","-23,803,763"


### Postpaid B2C

In [30]:
''' Postpaid SubBase B2C '''

v_metric_list = [
    'B2S010602' #Postpaid SubBase B2C
    , 'DB2S010602' #Postpaid SubBase B2C : DTAC
    , 'TB2S010602' #Postpaid SubBase B2C : TMH
    ]

postpaid_subbase_b2c_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
postpaid_subbase_b2c_df = postpaid_subbase_b2c_df.loc[postpaid_subbase_b2c_df['TM_KEY_MTH']>=v_tm_key_mth]
postpaid_subbase_b2c_df = postpaid_subbase_b2c_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
postpaid_subbase_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202506,Postpaid,B2S010602,Postpaid SubBase B2C,2025-06-11 01:34:02,20250608,0,"14,278,770","14,272,412","14,266,750","14,266,750","-14,278,770"
1,202506,Postpaid,DB2S010602,Postpaid SubBase B2C : DTAC,2025-06-11 01:34:02,20250609,0,"5,086,120","5,086,114","5,080,462","5,080,462","-5,086,120"
2,202506,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-06-11 01:34:02,20250608,0,"9,192,130","9,185,778","9,185,768","9,185,768","-9,192,130"


In [32]:
''' Postpaid 60DPD B2C '''

v_metric_list = [
    'B2S010601' #Postpaid 60DPD B2C
    , 'TB2S010601' #Postpaid 60DPD B2C : TMH
    , 'DB2S010601' #Postpaid 60DPD B2C : DTAC
    ]

postpaid_60dpd_b2c_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
postpaid_60dpd_b2c_df = postpaid_60dpd_b2c_df.loc[postpaid_60dpd_b2c_df['TM_KEY_MTH']>=v_tm_key_mth]
postpaid_60dpd_b2c_df = postpaid_60dpd_b2c_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
postpaid_60dpd_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202506,Postpaid,DB2S010601,Postpaid 60DPD B2C : DTAC,2025-06-11 01:34:02,20250609,0,"122,681","122,616","122,616","122,616","-122,681"
1,202506,Postpaid,TB2S010601,Postpaid 60DPD B2C : TMH,2025-06-11 01:34:02,20250608,0,"89,461","89,459","89,459","89,459","-89,461"


In [33]:
''' Postpaid Gain/Loss 60DPD B2C '''

v_metric_list = [
    'B2S010603' #Postpaid Gain/Loss 60DPD B2C
    , 'TB2S010603' #Postpaid Gain/Loss 60DPD B2C : TMH
    , 'DB2S010603' #Postpaid Gain/Loss 60DPD B2C : DTAC
    ]

postpaid_gl_60dpd_b2c_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
postpaid_gl_60dpd_b2c_df = postpaid_gl_60dpd_b2c_df.loc[postpaid_gl_60dpd_b2c_df['TM_KEY_MTH']>=v_tm_key_mth]
postpaid_gl_60dpd_b2c_df = postpaid_gl_60dpd_b2c_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
postpaid_gl_60dpd_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202506,Postpaid,DB2S010603,Postpaid Gain/Loss 60DPD B2C : DTAC,2025-06-11 01:34:02,20250608,"-3,104","-3,104","-3,104","-3,104","-3,104",0
1,202506,Postpaid,TB2S010603,Postpaid Gain/Loss 60DPD B2C : TMH,2025-06-11 01:34:02,20250608,"-2,985","-2,985","-2,985","-2,985","-2,985",0


In [34]:
''' Postpaid Reported SubBase B2C '''

v_metric_list = [
    'B2S010600' #Postpaid Reported SubBase B2C
    , 'DB2S010600' #Postpaid Reported SubBase B2C : DTAC
    , 'TB2S010600' #Postpaid Reported SubBase B2C : TMH
    ]

postpaid_reported_subbase_b2c_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
postpaid_reported_subbase_b2c_df = postpaid_reported_subbase_b2c_df.loc[postpaid_reported_subbase_b2c_df['TM_KEY_MTH']>=v_tm_key_mth]
postpaid_reported_subbase_b2c_df = postpaid_reported_subbase_b2c_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
postpaid_reported_subbase_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202506,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-06-11 01:34:02,20250608,0,"14,064,757","14,052,814","14,052,804","14,052,804","-14,064,757"
1,202506,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-06-11 01:34:02,20250608,0,"4,962,088","4,956,495","4,956,495","4,956,495","-4,962,088"
2,202506,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-06-11 01:34:02,20250608,0,"9,102,669","9,096,319","9,096,309","9,096,309","-9,102,669"


### FTTx

In [36]:
''' FTTx SubBase '''

v_metric_list = [
    'TB3S020604' #FTTx SubBase
    ]

fttx_subbase_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
fttx_subbase_df = fttx_subbase_df.loc[fttx_subbase_df['TM_KEY_MTH']>=v_tm_key_mth]
fttx_subbase_df = fttx_subbase_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
fttx_subbase_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202506,TOL,TB3S020604,FTTx SubBase,2025-06-11 01:34:02,20250608,0,"3,301,794","3,301,713","3,301,713","3,301,713","-3,301,794"


In [37]:
''' FTTx 60DPD '''

v_metric_list = [
    'TB3S020603' #FTTx 60DPD
    ]

fttx_60dpd_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
fttx_60dpd_df = fttx_60dpd_df.loc[fttx_60dpd_df['TM_KEY_MTH']>=v_tm_key_mth]
fttx_60dpd_df = fttx_60dpd_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
fttx_60dpd_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202506,TOL,TB3S020603,FTTx 60DPD,2025-06-11 01:34:02,20250608,0,"99,072","99,070","99,070","99,070","-99,072"


In [39]:
''' FTTx Gain/Loss 60DPD '''

v_metric_list = [
    'TB3S020606' #FTTx Gain/Loss 60DPD
    ]

fttx_gl_60dpd_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
fttx_gl_60dpd_df = fttx_gl_60dpd_df.loc[fttx_gl_60dpd_df['TM_KEY_MTH']>=v_tm_key_mth]
fttx_gl_60dpd_df = fttx_gl_60dpd_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
fttx_gl_60dpd_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202506,TOL,TB3S020606,FTTx Gain/Loss 60DPD,2025-06-11 01:34:02,20250608,0,-974,-974,-974,-974,974


In [40]:
''' FTTx Reported SubBase '''

v_metric_list = [
    'TB3S000600' #FTTx Reported SubBase
    ]

fttx_reported_subbase_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
fttx_reported_subbase_df = fttx_reported_subbase_df.loc[fttx_reported_subbase_df['TM_KEY_MTH']>=v_tm_key_mth]
fttx_reported_subbase_df = fttx_reported_subbase_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
fttx_reported_subbase_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202506,TOL,TB3S000600,FTTx Reported SubBase,2025-06-11 01:34:02,20250608,0,"3,202,722","3,202,643","3,202,643","3,202,643","-3,202,722"


## ** Current Issue

In [41]:
''' Issue : Prepaid SubBase : DTAC '''

v_metric_cd = 'DB1S000702'

issue_prepaid_subbase_dtac_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metric_cd]
issue_prepaid_subbase_dtac_df = issue_prepaid_subbase_dtac_df.loc[issue_prepaid_subbase_dtac_df['TM_KEY_MTH']>=202401]
issue_prepaid_subbase_dtac_df = issue_prepaid_subbase_dtac_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
issue_prepaid_subbase_dtac_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)


In [43]:
''' Issue : Prepaid Reported SubBase : DTAC '''

v_metric_cd = 'DB1S000700'

issue_prepaid_reported_subbase_dtac_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metric_cd]
issue_prepaid_reported_subbase_dtac_df = issue_prepaid_reported_subbase_dtac_df.loc[issue_prepaid_reported_subbase_dtac_df['TM_KEY_MTH']>=202401]
issue_prepaid_reported_subbase_dtac_df = issue_prepaid_reported_subbase_dtac_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
issue_prepaid_reported_subbase_dtac_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)


In [44]:
''' Issue : Postpaid SubBase B2C : TMH '''

v_metric_cd = 'TB2S010602'

issue_postpaid_subbase_b2c_true_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metric_cd]
issue_postpaid_subbase_b2c_true_df = issue_postpaid_subbase_b2c_true_df.loc[issue_postpaid_subbase_b2c_true_df['TM_KEY_MTH']>=202401]
issue_postpaid_subbase_b2c_true_df = issue_postpaid_subbase_b2c_true_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
issue_postpaid_subbase_b2c_true_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202402,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-06-11 01:34:02,20240229,0,"8,998,482","8,998,152","8,998,139","8,998,139","-8,998,482"
1,202403,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-06-11 01:34:02,20240331,0,"8,994,853",0,0,0,"-8,994,853"
2,202405,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-06-11 01:34:02,20240531,0,"9,023,324",0,0,0,"-9,023,324"
3,202406,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-06-11 01:34:02,20240630,0,"9,042,184","9,041,896","9,041,886","9,041,886","-9,042,184"
4,202407,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-06-11 01:34:02,20240731,0,"9,052,569","9,052,314","9,052,304","9,052,304","-9,052,569"
5,202408,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-06-11 01:34:02,20240831,0,"9,061,002","9,060,761","9,060,751","9,060,751","-9,061,002"
6,202409,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-06-11 01:34:02,20240930,0,"9,068,470","9,068,233","9,068,223","9,068,223","-9,068,470"
7,202410,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-06-11 01:34:02,20241031,0,"9,080,143","9,079,912","9,079,902","9,079,902","-9,080,143"
8,202411,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-06-11 01:34:02,20241130,0,"9,088,656","9,088,441","9,088,431","9,088,431","-9,088,656"
9,202412,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-06-11 01:34:02,20241231,0,"9,102,725","9,102,514","9,102,504","9,102,504","-9,102,725"


In [45]:
''' Issue : Postpaid SubBase B2C : DTAC '''

v_metric_cd = 'DB2S010602'

issue_postpaid_subbase_b2c_dtac_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metric_cd]
issue_postpaid_subbase_b2c_dtac_df = issue_postpaid_subbase_b2c_dtac_df.loc[issue_postpaid_subbase_b2c_dtac_df['TM_KEY_MTH']>=202401]
issue_postpaid_subbase_b2c_dtac_df = issue_postpaid_subbase_b2c_dtac_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
issue_postpaid_subbase_b2c_dtac_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202405,Postpaid,DB2S010602,Postpaid SubBase B2C : DTAC,2025-06-11 01:34:02,20240531,0,"5,372,517","5,372,508","5,176,569",0,"-5,372,517"
1,202406,Postpaid,DB2S010602,Postpaid SubBase B2C : DTAC,2025-06-11 01:34:02,20240630,0,"5,349,473","5,349,465","5,154,102",0,"-5,349,473"
2,202407,Postpaid,DB2S010602,Postpaid SubBase B2C : DTAC,2025-06-11 01:34:02,20240731,0,"5,319,937","5,319,931","5,126,018","5,306,861","-5,319,937"
3,202408,Postpaid,DB2S010602,Postpaid SubBase B2C : DTAC,2025-06-11 01:34:02,20240831,0,"5,292,344","5,292,338","5,098,221","5,279,274","-5,292,344"
4,202409,Postpaid,DB2S010602,Postpaid SubBase B2C : DTAC,2025-06-11 01:34:02,20240930,0,"5,269,557","5,269,550","5,076,200","5,256,503","-5,269,557"
5,202410,Postpaid,DB2S010602,Postpaid SubBase B2C : DTAC,2025-06-11 01:34:02,20241031,0,"5,241,249","5,241,245","5,049,529","5,228,437","-5,241,249"
6,202411,Postpaid,DB2S010602,Postpaid SubBase B2C : DTAC,2025-06-11 01:34:02,20241130,0,"5,204,919","5,204,916","5,020,614","5,199,257","-5,204,919"
7,202412,Postpaid,DB2S010602,Postpaid SubBase B2C : DTAC,2025-06-11 01:34:02,20241231,0,"5,184,717","5,184,712","5,179,054","5,179,054","-5,184,717"
8,202501,Postpaid,DB2S010602,Postpaid SubBase B2C : DTAC,2025-06-11 01:34:02,20250131,0,"5,161,625","5,161,620","5,155,964","5,155,964","-5,161,625"
9,202502,Postpaid,DB2S010602,Postpaid SubBase B2C : DTAC,2025-06-11 01:34:02,20250228,0,"5,138,667","5,138,664","5,133,010","5,133,010","-5,138,667"


In [46]:
''' Issue : Postpaid 60DPD B2C : TMH '''

v_metric_cd = 'TB2S010601'

issue_postpaid_60dpd_b2c_true_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metric_cd]
issue_postpaid_60dpd_b2c_true_df = issue_postpaid_60dpd_b2c_true_df.loc[issue_postpaid_60dpd_b2c_true_df['TM_KEY_MTH']>=202401]
issue_postpaid_60dpd_b2c_true_df = issue_postpaid_60dpd_b2c_true_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
issue_postpaid_60dpd_b2c_true_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202402,Postpaid,TB2S010601,Postpaid 60DPD B2C : TMH,2025-06-11 01:34:02,20240228,0,"98,379","98,376","98,376","98,376","-98,379"
1,202403,Postpaid,TB2S010601,Postpaid 60DPD B2C : TMH,2025-06-11 01:34:02,20240331,0,"79,598",0,0,0,"-79,598"
2,202405,Postpaid,TB2S010601,Postpaid 60DPD B2C : TMH,2025-06-11 01:34:02,20240531,0,"76,251",0,0,0,"-76,251"
3,202406,Postpaid,TB2S010601,Postpaid 60DPD B2C : TMH,2025-06-11 01:34:02,20240630,0,"81,727","81,725","81,725","81,725","-81,727"
4,202407,Postpaid,TB2S010601,Postpaid 60DPD B2C : TMH,2025-06-11 01:34:02,20240731,0,"83,942","83,940","83,940","83,940","-83,942"
5,202408,Postpaid,TB2S010601,Postpaid 60DPD B2C : TMH,2025-06-11 01:34:02,20240831,0,"86,053","86,053","86,053","86,053","-86,053"
6,202409,Postpaid,TB2S010601,Postpaid 60DPD B2C : TMH,2025-06-11 01:34:02,20240930,0,"85,778","85,777","85,777","85,777","-85,778"
7,202410,Postpaid,TB2S010601,Postpaid 60DPD B2C : TMH,2025-06-11 01:34:02,20241031,0,"85,630","85,630","85,630","85,630","-85,630"
8,202411,Postpaid,TB2S010601,Postpaid 60DPD B2C : TMH,2025-06-11 01:34:02,20241130,0,"86,769","86,769","86,769","86,769","-86,769"
9,202412,Postpaid,TB2S010601,Postpaid 60DPD B2C : TMH,2025-06-11 01:34:02,20241231,0,"80,142","80,142","80,142","80,142","-80,142"


In [47]:
''' Issue : Postpaid 60DPD B2C : DTAC '''

v_metric_cd = 'DB2S010601'

issue_postpaid_60dpd_b2c_dtac_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metric_cd]
issue_postpaid_60dpd_b2c_dtac_df = issue_postpaid_60dpd_b2c_dtac_df.loc[issue_postpaid_60dpd_b2c_dtac_df['TM_KEY_MTH']>=202401]
issue_postpaid_60dpd_b2c_dtac_df = issue_postpaid_60dpd_b2c_dtac_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
issue_postpaid_60dpd_b2c_dtac_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202405,Postpaid,DB2S010601,Postpaid 60DPD B2C : DTAC,2025-06-11 01:34:02,20240531,0,"171,764","171,687","167,980","171,687","-171,764"
1,202406,Postpaid,DB2S010601,Postpaid 60DPD B2C : DTAC,2025-06-11 01:34:02,20240630,0,"165,490","165,409","161,834","165,409","-165,490"
2,202407,Postpaid,DB2S010601,Postpaid 60DPD B2C : DTAC,2025-06-11 01:34:02,20240731,0,"163,473","163,393","159,897","163,393","-163,473"
3,202408,Postpaid,DB2S010601,Postpaid 60DPD B2C : DTAC,2025-06-11 01:34:02,20240831,0,"162,915","162,834","159,379","162,834","-162,915"
4,202409,Postpaid,DB2S010601,Postpaid 60DPD B2C : DTAC,2025-06-11 01:34:02,20240930,0,"151,412","151,333","148,074","151,333","-151,412"
5,202410,Postpaid,DB2S010601,Postpaid 60DPD B2C : DTAC,2025-06-11 01:34:02,20241031,0,"141,986","141,901","138,754","141,901","-141,986"
6,202411,Postpaid,DB2S010601,Postpaid 60DPD B2C : DTAC,2025-06-11 01:34:02,20241130,0,"133,053","132,985","129,965","132,985","-133,053"
7,202412,Postpaid,DB2S010601,Postpaid 60DPD B2C : DTAC,2025-06-11 01:34:02,20241231,0,"128,744","128,678","125,779","128,678","-128,744"
8,202501,Postpaid,DB2S010601,Postpaid 60DPD B2C : DTAC,2025-06-11 01:34:02,20250131,0,"127,686","127,622","124,809","127,622","-127,686"
9,202502,Postpaid,DB2S010601,Postpaid 60DPD B2C : DTAC,2025-06-11 01:34:02,20250228,0,"124,747","124,684","124,684","124,684","-124,747"


In [48]:
''' Issue : Postpaid Reported SubBase B2C : TMH '''

v_metric_cd = 'TB2S010600'

issue_postpaid_reported_subbase_b2c_true_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metric_cd]
issue_postpaid_reported_subbase_b2c_true_df = issue_postpaid_reported_subbase_b2c_true_df.loc[issue_postpaid_reported_subbase_b2c_true_df['TM_KEY_MTH']>=202401]
issue_postpaid_reported_subbase_b2c_true_df = issue_postpaid_reported_subbase_b2c_true_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
issue_postpaid_reported_subbase_b2c_true_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202402,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-06-11 01:34:02,20240229,0,"8,998,482","8,998,152","8,998,139","8,998,139","-8,998,482"
1,202403,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-06-11 01:34:02,20240331,0,"8,994,853",0,0,0,"-8,994,853"
2,202405,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-06-11 01:34:02,20240531,0,"9,023,324",0,0,0,"-9,023,324"
3,202406,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-06-11 01:34:02,20240630,0,"9,042,184","9,041,896","9,041,886","9,041,886","-9,042,184"
4,202407,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-06-11 01:34:02,20240731,0,"9,052,569","9,052,314","9,052,304","9,052,304","-9,052,569"
5,202408,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-06-11 01:34:02,20240831,0,"9,061,002","9,060,761","9,060,751","9,060,751","-9,061,002"
6,202409,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-06-11 01:34:02,20240930,0,"9,068,470","9,068,233","9,068,223","9,068,223","-9,068,470"
7,202410,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-06-11 01:34:02,20241031,0,"9,080,143","9,079,912","9,079,902","9,079,902","-9,080,143"
8,202411,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-06-11 01:34:02,20241130,0,"9,088,656","9,088,441","9,088,431","9,088,431","-9,088,656"
9,202412,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-06-11 01:34:02,20241231,0,"9,102,725","9,102,514","9,102,504","9,102,504","-9,102,725"


In [49]:
''' Issue : Postpaid Reported SubBase B2C : DTAC '''

v_metric_cd = 'DB2S010600'

issue_postpaid_reported_subbase_b2c_dtac_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metric_cd]
issue_postpaid_reported_subbase_b2c_dtac_df = issue_postpaid_reported_subbase_b2c_dtac_df.loc[issue_postpaid_reported_subbase_b2c_dtac_df['TM_KEY_MTH']>=202401]
issue_postpaid_reported_subbase_b2c_dtac_df = issue_postpaid_reported_subbase_b2c_dtac_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
issue_postpaid_reported_subbase_b2c_dtac_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202406,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-06-11 01:34:02,20240630,0,"5,183,983","5,170,964","4,992,268","5,170,964","-5,183,983"
1,202407,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-06-11 01:34:02,20240731,0,"5,156,464","5,143,468","4,966,121","5,143,468","-5,156,464"
2,202408,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-06-11 01:34:02,20240831,0,"5,129,429","5,116,440","4,938,842","5,116,440","-5,129,429"
3,202409,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-06-11 01:34:02,20240930,0,"5,118,145","5,105,170","4,928,126","5,105,170","-5,118,145"
4,202410,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-06-11 01:34:02,20241031,0,"5,099,263","5,086,536","4,910,775","5,086,536","-5,099,263"
5,202411,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-06-11 01:34:02,20241130,0,"5,071,866","5,066,272","4,890,649","5,066,272","-5,071,866"
6,202412,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-06-11 01:34:02,20241231,0,"5,055,973","5,050,376","4,874,826","5,050,376","-5,055,973"
7,202501,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-06-11 01:34:02,20250131,0,"5,033,939","5,028,342","4,854,928","5,028,342","-5,033,939"
8,202502,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-06-11 01:34:02,20250228,0,"5,013,920","5,008,326","5,008,326","5,008,326","-5,013,920"
9,202503,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-06-11 01:34:02,20250331,0,"4,997,992","4,992,399","4,992,399","4,992,399","-4,997,992"
